# Try to analyze how good the reattachment method is for the case of heptane

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/home/yentingw/AutoFragmentModeling/ipython/2. simulation/heptane/simulation_1_202.csv')

In [3]:
data_1 = list(data.head(0))

In [4]:
index = data.__len__()
value = data.loc[index-1]
data_2 = value.tolist()

In [5]:
VData = data.iloc[:,1]
P = 40 # bar
T = 1600 # K

In [6]:
total_moles = P*100000*VData/8.314/T # mol

In [7]:
moles_dict ={}
for key, val in zip(data_1[6:-1], data_2[6:-1]):
    moles_dict[key] = val

In [8]:
moles_dict['rcccc(2)']

0.0019492016834161355

In [9]:
with open('/home/yentingw/AutoFragmentModeling/examples/heptane/species_dictionary.txt', 'r') as f:
    species_dict = f.read()
    splat = species_dict.split("\n\n")

In [10]:
	r_moles = []
	l_moles = []
	r_l_moles = []
	rr_ll_list = []
	remain_moles = []
label_dict = {}

In [11]:
for key in moles_dict:
    for paragraph in splat:
        Rcount = 0
        Lcount = 0
        all_line = paragraph.split('\n')
        if key == all_line[0]:
            for line in all_line:
                if line[0].isalpha() == False:
                    if 'R' in line:
                        Rcount += 1
                    if 'L' in line:
                        Lcount += 1
            label_dict[key] = (Rcount, Lcount)

In [ ]:
label_dict

In [12]:
	for spe_label in moles_dict:
        r_count = label_dict[spe_label][0]
        l_count = label_dict[spe_label][1]
		label_count = r_count + l_count
		if label_count == 0:
			remain_moles.append((spe_label, moles_dict[spe_label]))
			continue

		if label_count == 1:
			if r_count == 1:
				r_moles.append((spe_label, moles_dict[spe_label]))
			elif l_count == 1:
				l_moles.append((spe_label, moles_dict[spe_label]))
		elif label_count == 2 and l_count == r_count:
			r_l_moles.append((spe_label, moles_dict[spe_label]))
		elif label_count == 2 and l_count != r_count:
			rr_ll_list.append(spe_label)
			if r_count == 2:
				r_moles.append((spe_label, 2*moles_dict[spe_label]))
			elif l_count == 2:
				l_moles.append((spe_label, 2*moles_dict[spe_label]))
		else:
			print "{0} has more than two cutting labels, which is not supported.".format(spe_label)


In [13]:
from afm.simulator import OdeSimulator
import afm.utils
import afm.simulator
import afm.fragment
from afm.simulator import categorize_fragments

In [ ]:
moles_dict

In [14]:
for key in moles_dict:
    moles_dict[key] = moles_dict[key] * total_moles[index-1] # value in unit of mol

In [ ]:
moles_dict

In [15]:
grind_size = 1e-6
shuffle_seed = 0

In [16]:
		# cut large moles into smaller pieces
		grinded_r_moles = afm.utils.grind(r_moles, grind_size)
		grinded_l_moles = afm.utils.grind(l_moles, grind_size)

		# random shuffle
		r_moles_shuffle = afm.utils.shuffle(grinded_r_moles, shuffle_seed)
		l_moles_shuffle = afm.utils.shuffle(grinded_l_moles, shuffle_seed)

		# match concentrations for single-labeled fragments
		# including RCCCCR
		matches0 = afm.utils.match_concentrations_with_same_sums(l_moles_shuffle, 
													   			 r_moles_shuffle, 
													   			 diff_tol=1e-3)

		matches1, new_r_l_moles = afm.utils.matches_resolve(matches0, rr_ll_list)

		r_l_moles.extend(new_r_l_moles)
		# insert double-labeled fragments into matches
		# e.g., LCCCCR
		matches = afm.utils.match_concentrations_with_different_sums(matches1, r_l_moles)

In [17]:
		flattened_matches = [(tuple(afm.utils.flatten(m[0])), m[1]) for m in matches]

		final_frags_moles = []
		for remain in remain_moles:
			label, val = remain
			final_frags_moles.append(((label, ), val))

		final_frags_moles.extend(flattened_matches)

In [ ]:
final_frags_moles

In [18]:
frag_weight_dict = {}
for key in moles_dict:
    for paragraph in splat:
        all_line = paragraph.split('\n')
        if key == all_line[0]:
            adj = afm.fragment.Fragment().fromAdjacencyList(paragraph)
            fragmental_weight = adj.getMolecularWeight()
            frag_weight_dict[key] = fragmental_weight

In [ ]:
frag_weight_dict

In [19]:
		# calculate fragmental weight distribution
		fragmental_weight_distri = []
		for final_frag_mole in final_frags_moles:
			sub_frag_labels, mole = final_frag_mole
			total_frag_weight = 0
			for sub_frag_label in sub_frag_labels:
				sub_frag = frag_weight_dict[sub_frag_label]
				total_frag_weight += sub_frag
			
			fragmental_weight_distri.append((total_frag_weight, mole)) # (reattached-fragment weight, number of moles)

In [20]:
fragmental_weight_distri

[(0.028053160291165113, 0.14730390158863418),
 (0.043087680474855006, 1.3721377923494606e-06),
 (0.043087680474855006, 4.316010558161749e-06),
 (0.080127720721066, 2.660064624409733e-05),
 (0.041071800398640335, 1.618135637992977e-06),
 (0.08214360079728067, 3.985630073541468e-05),
 (0.06912496068980545, 5.69295776160189e-08),
 (0.05711426062043756, 1.0183566360544632e-07),
 (0.002015880076214671, 0.266816717047969),
 (0.041071800398640335, 0.0003435188697266822),
 (0.04207974043674767, 0.022157399529509497),
 (0.02906110032927245, 9.821334466783243e-05),
 (0.0010079400381073356, 1.8622624282002492e-05),
 (0.06710908061359078, 1.8780711352909308e-08),
 (0.08113566075917333, 2.4351582249348177e-09),
 (0.05509838054422289, 4.648654146959457e-07),
 (0.027045220253057778, 2.0845085897131518e-05),
 (0.040063860360533, 0.0030878263472079),
 (0.040063860360533, 0.0032255012077208093),
 (0.016042460221797228, 0.13422548363152026),
 (0.039055920322425663, 0.0002921030135963103),
 (0.06509320053

In [29]:
mws = [tup[0]*1000 for tup in fragmental_weight_distri]
moles = [tup[1] for tup in fragmental_weight_distri]
molefracs = moles/sum(moles)

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [38]:
tot = sum(moles)

In [36]:
moles

[0.14730390158863418,
 1.3721377923494606e-06,
 4.316010558161749e-06,
 2.660064624409733e-05,
 1.618135637992977e-06,
 3.985630073541468e-05,
 5.69295776160189e-08,
 1.0183566360544632e-07,
 0.266816717047969,
 0.0003435188697266822,
 0.022157399529509497,
 9.821334466783243e-05,
 1.8622624282002492e-05,
 1.8780711352909308e-08,
 2.4351582249348177e-09,
 4.648654146959457e-07,
 2.0845085897131518e-05,
 0.0030878263472079,
 0.0032255012077208093,
 0.13422548363152026,
 0.0002921030135963103,
 1.4838094946166055e-09,
 0.00016606310268992304,
 7.774873178834425e-08,
 2.4130397442360566e-07,
 0.001252513845971052,
 1.2519105857677896e-08,
 0.007064393392229701,
 0.00201014296422738,
 0.00032784861541933463,
 0.0009071897697309489,
 0.0004394826150393431,
 3.667773491141293e-05,
 1.2033643854662204e-08,
 0.0003807979836037287,
 0.02545862417783924,
 5e-07,
 1e-06,
 1.0000000000000002e-06,
 9.999999999999997e-07,
 9.999999999999997e-07,
 9.999999999999997e-07,
 9.999999999999997e-07,
 1.000

In [41]:
molefracs = [mol/tot for mol in moles]

In [43]:
molefracs

[0.18480472045439283,
 1.7214584162760259e-06,
 5.414786139926883e-06,
 3.3372673364585945e-05,
 2.030082713434769e-06,
 5.000296961803519e-05,
 7.142278353424693e-08,
 1.2776111930455616e-07,
 0.33474326392460857,
 0.0004309723503993518,
 0.027798259122032973,
 0.00012321662570022627,
 2.336359619430613e-05,
 2.356192927734948e-08,
 3.055103973267862e-09,
 5.8321145662328e-07,
 2.6151854978187715e-05,
 0.0038739291950397005,
 0.004046653500616742,
 0.16839678184255744,
 0.0003664669787383081,
 1.8615596457585874e-09,
 0.00020833966337228577,
 9.754210505539773e-08,
 3.0273545409831315e-07,
 0.0015713804500328097,
 1.5706236110623354e-08,
 0.008862855850734664,
 0.002521887774668888,
 0.0004113127423680187,
 0.0011381433212978042,
 0.0005513666708144957,
 4.601520037217392e-05,
 1.5097184559435746e-08,
 0.00047774202957644857,
 0.031939913835262936,
 6.272906503538674e-07,
 1.2545813007077347e-06,
 1.254581300707735e-06,
 1.2545813007077343e-06,
 1.2545813007077343e-06,
 1.254581300707

In [46]:
import os
working_dir = os.path.join('/home/yentingw/AutoFragmentModeling/examples/heptane')
results_path = os.path.join(working_dir, 'results')
if not os.path.exists(results_path):
    os.mkdir(results_path)

In [48]:
example = 'heptane'
import numpy
numpy.savetxt(os.path.join(results_path, 'mwd_{0}.csv'.format(example)), (mws, molefracs))